In [ ]:
%reset -f 
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=

import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras import utils as np_utils
import numpy as np
from tqdm import tqdm
from collections import namedtuple, defaultdict
from sklearn.model_selection import train_test_split
import random

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
from scipy import signal
import json
import os

tf.config.list_physical_devices('GPU')


# Dataloader

In [ ]:
trial_info = namedtuple('trial_structure', ['subject', 'task', 'stim', 'epoch', 'label'])
eeg_load_trial = namedtuple('eeg_preload_trial', ['data', 'label'])

class SpeechDataset:

    def __init__(self, filename, points, shuffle_labels=False, subjects=None, tasks=None, stims=None, trials=None):
        self.filename = filename
        self.subjects = subjects
        self.tasks = tasks
        self.stims = stims
        self.srate = 1000
        self.points = points
        self.shuffle_labels = shuffle_labels

        

        if trials is None:
            self.stim2id = {stim_value: index+1 for index, stim_value in enumerate(self.stims)}
            self.get_info()

        else:
            self.trials = trials



    def get_info(self):
        self.trials = []  # Trials will be stored here

        with np.load(self.filename, 'r') as data:
            print("Loading subjects info")
            # Iterate over subjects, conditions and stims
            for key in tqdm(data.keys()):
                subject, task, stim = key.split('_')
                stim = int(stim)  # convert to int

                # Filter by subject, task and stim according to the inputs 
                # * Subject
                if self.subjects is not None:
                    if subject not in self.subjects:  # If subject not in self.subject continue
                        continue
                
                # * Task
                if self.tasks is not None:
                    if task not in self.tasks:
                        continue
                
                # * stim
                if self.stims is not None:
                    if stim not in self.stims:
                        continue

                # Iterate over repetitions for a stimulus
                num_epoch, _, _ = data[key].shape
                for epoch_index in range(num_epoch):
                    self.trials.append(
                        trial_info(
                            subject=subject,
                            task=task,
                            stim=stim,
                            label=self.stim2id[stim],
                            epoch=epoch_index,
                        )
                    )

    def load_eeg(self):
        self.X = np.zeros((len(self.trials), 14, self.points, 1))
        self.Y = np.zeros((len(self.trials)))
        

        with np.load(self.filename) as data:
            print("Loading eegdata into memory")
            for idx, single_trial in tqdm(enumerate(self.trials), total=len(self.trials)):
                subject = single_trial.subject
                task = single_trial.task
                stim = single_trial.stim
                epoch = single_trial.epoch
                label = single_trial.label

                text =  f"{subject}_{task}_{stim}"
                self.X[idx, :, :, 0] = data[text][epoch][:, :self.points][:, :]  # Only 3 seconds
                if self.shuffle_labels:
                    self.Y[idx] = random.randint(1, len(self.stims))
                else:
                    self.Y[idx] = label
             

        return self.X, self.Y        
    
    def resample_augmentation(self, X, Y, factor=4, test=False):
        num_samples, num_channels, num_points, *_  = X.shape
        
        new_X = np.zeros((num_samples * factor, num_channels, int(num_points/factor), 1))
        new_Y = np.zeros((num_samples * factor))
        idx = 0
        for i in range(num_samples):
            for f in range(factor):
                new_X[idx, :, :, :] = X[i, :, f::factor, :]
                new_Y[idx] = Y[i]
                idx+=1
                if test:
                    break
        X = new_X
        Y = new_Y
        return X, Y

    def balanced_split(self, takes=2, shuffle_labels=False):
        # Split sets into train and test
        
        to_split = defaultdict(list)
        
        for trial in self.trials:
            subject = trial.subject
            task = trial.subject
            stim = trial.stim
            
            to_split[f"{subject}_{task}_{stim}"].append(trial)
        
        train = []
        test = []
        
        for trial_list in to_split.values():
            indexs = list(range(len(trial_list)))
            
            test_index = random.sample(indexs, takes)
            train_index = [i for i in indexs if i not in test_index]

            
            # add for test
            for t in test_index:
                test.append(trial_list[t])
                
            # add for train
            for t in train_index:
                train.append(trial_list[t])
        
            
        train_dataset = SpeechDataset(self.filename, self.points, shuffle_labels=shuffle_labels,stims=self.stims, trials=train)
        test_dataset = SpeechDataset(self.filename, self.points, shuffle_labels=shuffle_labels, stims=self.stims, trials=test)
                    
        X_train, Y_train = train_dataset.load_eeg()
        X_test, Y_test = test_dataset.load_eeg()
    
        
        return X_train, Y_train, X_test, Y_test

        




dataset_folder ='/tf/seagate/speech_datasets/numpy/language_average_2-50hz_ica_eyes_60sessions.npz'
speech_dataset = SpeechDataset(dataset_folder, points=100,stims=list(range(1, 6)), tasks=['perception'], subjects=['S8'])
X_train, Y_train, X_test, Y_test = speech_dataset.balanced_split(takes=1, shuffle_labels=True)

print(Y_test)
print(X_train.shape)




# Models

In [ ]:
def EEGNet(nb_classes, Chans = 64, Samples = 128, 
             dropoutRate = 0.5, kernLength = 500, F1 = 8, 
             D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):
    """ Keras Implementation of EEGNet
    http://iopscience.iop.org/article/10.1088/1741-2552/aace8c/meta
    Note that this implements the newest version of EEGNet and NOT the earlier
    version (version v1 and v2 on arxiv). We strongly recommend using this
    architecture as it performs much better and has nicer properties than
    our earlier version. For example:
        
        1. Depthwise Convolutions to learn spatial filters within a 
        temporal convolution. The use of the depth_multiplier option maps 
        exactly to the number of spatial filters learned within a temporal
        filter. This matches the setup of algorithms like FBCSP which learn 
        spatial filters within each filter in a filter-bank. This also limits 
        the number of free parameters to fit when compared to a fully-connected
        convolution. 
        
        2. Separable Convolutions to learn how to optimally combine spatial
        filters across temporal bands. Separable Convolutions are Depthwise
        Convolutions followed by (1x1) Pointwise Convolutions. 
        
    
    While the original paper used Dropout, we found that SpatialDropout2D 
    sometimes produced slightly better results for classification of ERP 
    signals. However, SpatialDropout2D significantly reduced performance 
    on the Oscillatory dataset (SMR, BCI-IV Dataset 2A). We recommend using
    the default Dropout in most cases.
        
    Assumes the input signal is sampled at 128Hz. If you want to use this model
    for any other sampling rate you will need to modify the lengths of temporal
    kernels and average pooling size in blocks 1 and 2 as needed (double the 
    kernel lengths for double the sampling rate, etc). Note that we haven't 
    tested the model performance with this rule so this may not work well. 
    
    The model with default parameters gives the EEGNet-8,2 model as discussed
    in the paper. This model should do pretty well in general, although it is
	advised to do some model searching to get optimal performance on your
	particular dataset.
    We set F2 = F1 * D (number of input filters = number of output filters) for
    the SeparableConv2D layer. We haven't extensively tested other values of this
    parameter (say, F2 < F1 * D for compressed learning, and F2 > F1 * D for
    overcomplete). We believe the main parameters to focus on are F1 and D. 
    Inputs:
        
      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer. We found
                        that setting this to be half the sampling rate worked
                        well in practice. For the SMR dataset in particular
                        since the data was high-passed at 4Hz we used a kernel
                        length of 32.     
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. Default: F1 = 8, F2 = F1 * D. 
      D               : number of spatial filters to learn within each temporal
                        convolution. Default: D = 2
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.
    """
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16*2),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(nb_classes, name = 'dense', 
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)



In [ ]:

dataset_folder ='language_average_2-50hz_icaLabel95confidence_eyes_60sessions.npz'

all_subjects = ['S2', 'S3', 'S5', 'S6', 'S8', 'S9', 
                'S10', 'S11', 'S12', 'S13', 'S14', 
                'S15', 'S16', 'S17', 'S18', 'S19', 
                'S21', 'S22', 'S23', 'S24', 'S25', 
                'S26', 'S27', 'S28', 'S29', 'S30', 
                'S31', 'S32', 'S33', 'S34', 'S35', 
                'S36', 'S37', 'S38', 'S39', 'S40', 
                'S41', 'S42', 'S43', 'S44', 'S46', 
                'S47', 'S48', 'S49', 'S50', 'S51', 
                'S52', 'S53', 'S55', 'S58', 'S59', 
                'S60', 'S61', 'S62', 'SS57a', 'SS57b', 
                'SS63a', 'SS63b', 'SS63c', 'SS63d']


# Effects of amount of data
from IPython.display import clear_output


results = defaultdict(list)
for RR in tqdm(range(100)):
    for points in [ 50, 125.,  250.,  375.,  500.,  625.,  750.,  875., 1000.]:
        print(RR, int(points))
        points = int(points)
        ### Load data ###
        speech_dataset = SpeechDataset(dataset_folder, points, shuffle_labels = True, stims=list(range(1, 31)), tasks=['perception'], subjects=all_subjects)
        X_train, Y_train, X_test, Y_test = speech_dataset.balanced_split(takes=1, shuffle_labels=False)


        # One-hot encoding of labels
        Y_train = np_utils.to_categorical(Y_train-1)
        Y_test = np_utils.to_categorical(Y_test-1)

        ### model ###

        model = EEGNet(30, Chans = 14, Samples = points, 
                     dropoutRate = 0.25, kernLength = 125, F1 = 8, 
                     D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout')
        #model.summary()
        #loss = tf.keras.losses.CosineSimilarity(axis=1)
        loss = 'categorical_crossentropy'
        model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
                      metrics = ['accuracy'])


        ### Training ###
        # Early stopping
        early_stopping_callbcak = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            min_delta=0.00001,
            patience=5
        )


        history = model.fit(X_train, Y_train, 
                            batch_size = 32, 
                            epochs = 100, 
                            verbose = 1, 
                            validation_data=(X_test, Y_test), 
                            callbacks=[early_stopping_callbcak])

        results[points].append(history.history['val_accuracy'][-5]) 
        clear_output(wait=True)
        
    np.save('30stim_results', results) 


results = defaultdict(list)
for RR in tqdm(range(100)):
    for points in [ 50, 125.,  250.,  375.,  500.,  625.,  750.,  875., 1000.]:
        print(RR, int(points))
        points = int(points)
        ### Load data ###
        speech_dataset = SpeechDataset(dataset_folder, points, shuffle_labels = True, stims=list(range(1, 6)), tasks=['perception'], subjects=all_subjects)
        X_train, Y_train, X_test, Y_test = speech_dataset.balanced_split(takes=1, shuffle_labels=False)


        # One-hot encoding of labels
        Y_train = np_utils.to_categorical(Y_train-1)
        Y_test = np_utils.to_categorical(Y_test-1)

        ### model ###

        model = EEGNet(5, Chans = 14, Samples = points, 
                     dropoutRate = 0.25, kernLength = 125, F1 = 8, 
                     D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout')
        #model.summary()
        #loss = tf.keras.losses.CosineSimilarity(axis=1)
        loss = 'categorical_crossentropy'
        model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
                      metrics = ['accuracy'])


        ### Training ###
        # Early stopping
        early_stopping_callbcak = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            min_delta=0.00001,
            patience=5
        )


        history = model.fit(X_train, Y_train, 
                            batch_size = 32, 
                            epochs = 100, 
                            verbose = 1, 
                            validation_data=(X_test, Y_test), 
                            callbacks=[early_stopping_callbcak])

        results[points].append(history.history['val_accuracy'][-5]) 
        clear_output(wait=True)
        
    np.save('5stim_results', results) 
    

